# Trading Bot Performance Analysis

This notebook provides comprehensive performance analysis for the trading bot based on trade execution logs and portfolio snapshots.

## Features:
- Load and analyze trade data from `trades.csv` and `portfolio_history.csv`
- Calculate key performance metrics (Sharpe ratio, max drawdown, profit factor, etc.)
- Generate comprehensive visualizations
- Trade distribution analysis
- Portfolio performance tracking

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from pathlib import Path
from typing import Dict, List, Tuple, Optional

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## 1. Data Loading Functions

In [ ]:
def load_trades_data(filepath: str = "../logs/trades.csv") -> pd.DataFrame:
    """
    Load and preprocess trade data from CSV file.
    
    Args:
        filepath: Path to trades CSV file
        
    Returns:
        Processed DataFrame with trade data
    """
    try:
        # Load the CSV file
        df = pd.read_csv(filepath)
        
        # Convert timestamp columns to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['execution_time'] = pd.to_datetime(df['execution_time'])
        
        # Ensure numeric columns are properly typed
        numeric_columns = ['entry_price', 'exit_price', 'size', 'pnl', 'fees']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Add derived columns
        df['trade_value'] = df['size'] * df['entry_price']
        df['gross_pnl'] = df['pnl'] + df['fees']  # PnL before fees
        df['return_pct'] = (df['exit_price'] - df['entry_price']) / df['entry_price'] * 100
        df['date'] = df['timestamp'].dt.date
        df['hour'] = df['timestamp'].dt.hour
        df['day_of_week'] = df['timestamp'].dt.day_name()
        
        # Classify trades as winners/losers
        df['trade_result'] = df['pnl'].apply(lambda x: 'Win' if x > 0 else 'Loss' if x < 0 else 'Breakeven')
        
        print(f"Loaded {len(df)} trades from {filepath}")
        print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
        print(f"Symbols: {df['symbol'].unique()}")
        
        return df
        
    except Exception as e:
        print(f"Error loading trades data: {e}")
        return pd.DataFrame()


def load_portfolio_data(filepath: str = "../logs/portfolio_history.csv") -> pd.DataFrame:
    """
    Load and preprocess portfolio history data from CSV file.
    
    Args:
        filepath: Path to portfolio history CSV file
        
    Returns:
        Processed DataFrame with portfolio data
    """
    try:
        # Load the CSV file
        df = pd.read_csv(filepath)
        
        # Convert timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Ensure numeric columns are properly typed
        numeric_columns = ['total_balance', 'unrealized_pnl', 'open_positions_count']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Sort by timestamp
        df = df.sort_values('timestamp').reset_index(drop=True)
        
        # Add derived columns
        df['date'] = df['timestamp'].dt.date
        df['hour'] = df['timestamp'].dt.hour
        
        # Calculate returns
        df['balance_change'] = df['total_balance'].diff()
        df['balance_change_pct'] = df['total_balance'].pct_change() * 100
        
        print(f"Loaded {len(df)} portfolio snapshots from {filepath}")
        print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
        print(f"Balance range: ${df['total_balance'].min():.2f} to ${df['total_balance'].max():.2f}")
        
        return df
        
    except Exception as e:
        print(f"Error loading portfolio data: {e}")
        return pd.DataFrame()


# Load the data
trades_df = load_trades_data()
portfolio_df = load_portfolio_data()

# Display basic info
if not trades_df.empty:
    print("\nTrades Data Sample:")
    display(trades_df.head())
    
if not portfolio_df.empty:
    print("\nPortfolio Data Sample:")
    display(portfolio_df.head())

## 2. Key Performance Metrics Calculation

In [ ]:
class PerformanceCalculator:
    """
    Calculate comprehensive trading performance metrics.
    """
    
    def __init__(self, trades_df: pd.DataFrame, portfolio_df: pd.DataFrame):
        self.trades_df = trades_df
        self.portfolio_df = portfolio_df
        self.risk_free_rate = 0.02  # 2% annual risk-free rate
        
    def calculate_basic_metrics(self) -> Dict[str, float]:
        """
        Calculate basic trading metrics.
        """
        if self.trades_df.empty:
            return {}
            
        total_trades = len(self.trades_df)
        winning_trades = len(self.trades_df[self.trades_df['pnl'] > 0])
        losing_trades = len(self.trades_df[self.trades_df['pnl'] < 0])
        
        total_pnl = self.trades_df['pnl'].sum()
        total_fees = self.trades_df['fees'].sum()
        gross_pnl = total_pnl + total_fees
        
        avg_win = self.trades_df[self.trades_df['pnl'] > 0]['pnl'].mean() if winning_trades > 0 else 0
        avg_loss = self.trades_df[self.trades_df['pnl'] < 0]['pnl'].mean() if losing_trades > 0 else 0
        
        largest_win = self.trades_df['pnl'].max()
        largest_loss = self.trades_df['pnl'].min()
        
        # Win rate
        win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
        
        # Profit factor
        gross_profit = self.trades_df[self.trades_df['pnl'] > 0]['pnl'].sum()
        gross_loss = abs(self.trades_df[self.trades_df['pnl'] < 0]['pnl'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        # Expectancy
        expectancy = (win_rate/100 * avg_win) + ((100-win_rate)/100 * avg_loss)
        
        return {
            'total_trades': total_trades,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades,
            'win_rate': win_rate,
            'total_pnl': total_pnl,
            'gross_pnl': gross_pnl,
            'total_fees': total_fees,
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'largest_win': largest_win,
            'largest_loss': largest_loss,
            'profit_factor': profit_factor,
            'expectancy': expectancy
        }
    
    def calculate_equity_curve(self) -> pd.DataFrame:
        """
        Calculate equity curve from trades.
        """
        if self.trades_df.empty:
            return pd.DataFrame()
            
        # Sort trades by timestamp
        trades_sorted = self.trades_df.sort_values('timestamp').copy()
        
        # Calculate cumulative PnL
        trades_sorted['cumulative_pnl'] = trades_sorted['pnl'].cumsum()
        
        # Assume starting balance
        starting_balance = 10000
        trades_sorted['equity'] = starting_balance + trades_sorted['cumulative_pnl']
        
        return trades_sorted[['timestamp', 'cumulative_pnl', 'equity']]
    
    def calculate_drawdown(self, equity_curve: pd.DataFrame) -> Dict[str, float]:
        """
        Calculate maximum drawdown and related metrics.
        """
        if equity_curve.empty:
            return {}
            
        # Calculate running maximum (peak)
        equity_curve = equity_curve.copy()
        equity_curve['peak'] = equity_curve['equity'].cummax()
        
        # Calculate drawdown
        equity_curve['drawdown'] = (equity_curve['equity'] - equity_curve['peak']) / equity_curve['peak'] * 100
        
        max_drawdown = equity_curve['drawdown'].min()
        max_drawdown_idx = equity_curve['drawdown'].idxmin()
        max_drawdown_date = equity_curve.loc[max_drawdown_idx, 'timestamp']
        
        # Calculate average drawdown
        avg_drawdown = equity_curve[equity_curve['drawdown'] < 0]['drawdown'].mean()
        avg_drawdown = avg_drawdown if not pd.isna(avg_drawdown) else 0
        
        return {
            'max_drawdown': max_drawdown,
            'max_drawdown_date': max_drawdown_date,
            'avg_drawdown': avg_drawdown,
            'drawdown_series': equity_curve['drawdown']
        }
    
    def calculate_sharpe_ratio(self, equity_curve: pd.DataFrame) -> float:
        """
        Calculate Sharpe ratio from equity curve.
        """
        if equity_curve.empty or len(equity_curve) < 2:
            return 0
            
        # Calculate daily returns
        equity_curve = equity_curve.copy()
        equity_curve['daily_return'] = equity_curve['equity'].pct_change()
        
        # Remove NaN values
        daily_returns = equity_curve['daily_return'].dropna()
        
        if len(daily_returns) < 2:
            return 0
            
        # Calculate average return and standard deviation
        avg_return = daily_returns.mean()
        std_return = daily_returns.std()
        
        if std_return == 0:
            return 0
            
        # Convert risk-free rate to daily
        daily_rf_rate = self.risk_free_rate / 365
        
        # Calculate Sharpe ratio
        sharpe_ratio = (avg_return - daily_rf_rate) / std_return
        
        # Annualize
        return sharpe_ratio * np.sqrt(365)
    
    def calculate_portfolio_metrics(self) -> Dict[str, float]:
        """
        Calculate metrics from portfolio history.
        """
        if self.portfolio_df.empty:
            return {}
            
        starting_balance = self.portfolio_df['total_balance'].iloc[0]
        ending_balance = self.portfolio_df['total_balance'].iloc[-1]
        
        total_return = ((ending_balance - starting_balance) / starting_balance) * 100
        
        # Calculate time period
        start_date = self.portfolio_df['timestamp'].iloc[0]
        end_date = self.portfolio_df['timestamp'].iloc[-1]
        days = (end_date - start_date).days
        
        # Annualized return
        annualized_return = ((ending_balance / starting_balance) ** (365 / days) - 1) * 100 if days > 0 else 0
        
        # Volatility
        volatility = self.portfolio_df['balance_change_pct'].std() * np.sqrt(365) if not self.portfolio_df['balance_change_pct'].isna().all() else 0
        
        return {
            'starting_balance': starting_balance,
            'ending_balance': ending_balance,
            'total_return': total_return,
            'annualized_return': annualized_return,
            'volatility': volatility,
            'days_tracked': days
        }


# Initialize calculator and compute metrics
calculator = PerformanceCalculator(trades_df, portfolio_df)

# Calculate all metrics
basic_metrics = calculator.calculate_basic_metrics()
equity_curve = calculator.calculate_equity_curve()
drawdown_metrics = calculator.calculate_drawdown(equity_curve)
sharpe_ratio = calculator.calculate_sharpe_ratio(equity_curve)
portfolio_metrics = calculator.calculate_portfolio_metrics()

print("Performance metrics calculated successfully!")

## 3. Performance Summary Report

In [ ]:
def print_performance_report():
    """
    Print comprehensive performance report.
    """
    print("="*80)
    print("                    TRADING PERFORMANCE REPORT")
    print("="*80)
    
    # Basic Trading Metrics
    print("\n📊 BASIC TRADING METRICS")
    print("-" * 40)
    if basic_metrics:
        print(f"Total Trades:        {basic_metrics['total_trades']:,}")
        print(f"Winning Trades:      {basic_metrics['winning_trades']:,} ({basic_metrics['win_rate']:.1f}%)")
        print(f"Losing Trades:       {basic_metrics['losing_trades']:,}")
        print(f"Win Rate:            {basic_metrics['win_rate']:.1f}%")
        print(f"Profit Factor:       {basic_metrics['profit_factor']:.2f}")
        print(f"Expectancy:          ${basic_metrics['expectancy']:.2f}")
    
    # Financial Performance
    print("\n💰 FINANCIAL PERFORMANCE")
    print("-" * 40)
    if basic_metrics:
        print(f"Total P&L:           ${basic_metrics['total_pnl']:.2f}")
        print(f"Gross P&L:           ${basic_metrics['gross_pnl']:.2f}")
        print(f"Total Fees:          ${basic_metrics['total_fees']:.2f}")
        print(f"Average Win:         ${basic_metrics['avg_win']:.2f}")
        print(f"Average Loss:        ${basic_metrics['avg_loss']:.2f}")
        print(f"Largest Win:         ${basic_metrics['largest_win']:.2f}")
        print(f"Largest Loss:        ${basic_metrics['largest_loss']:.2f}")
    
    # Portfolio Metrics
    print("\n📈 PORTFOLIO PERFORMANCE")
    print("-" * 40)
    if portfolio_metrics:
        print(f"Starting Balance:    ${portfolio_metrics['starting_balance']:,.2f}")
        print(f"Ending Balance:      ${portfolio_metrics['ending_balance']:,.2f}")
        print(f"Total Return:        {portfolio_metrics['total_return']:.2f}%")
        print(f"Annualized Return:   {portfolio_metrics['annualized_return']:.2f}%")
        print(f"Volatility:          {portfolio_metrics['volatility']:.2f}%")
        print(f"Days Tracked:        {portfolio_metrics['days_tracked']}")
    
    # Risk Metrics
    print("\n⚠️  RISK METRICS")
    print("-" * 40)
    if drawdown_metrics:
        print(f"Max Drawdown:        {drawdown_metrics['max_drawdown']:.2f}%")
        if 'max_drawdown_date' in drawdown_metrics:
            print(f"Max DD Date:         {drawdown_metrics['max_drawdown_date']}")
        print(f"Avg Drawdown:        {drawdown_metrics['avg_drawdown']:.2f}%")
    print(f"Sharpe Ratio:        {sharpe_ratio:.2f}")
    
    # Performance Rating
    print("\n🏆 PERFORMANCE RATING")
    print("-" * 40)
    
    rating_score = 0
    if basic_metrics and basic_metrics['win_rate'] > 50:
        rating_score += 1
    if basic_metrics and basic_metrics['profit_factor'] > 1.5:
        rating_score += 1
    if portfolio_metrics and portfolio_metrics['total_return'] > 10:
        rating_score += 1
    if sharpe_ratio > 1.0:
        rating_score += 1
    if drawdown_metrics and drawdown_metrics['max_drawdown'] > -10:
        rating_score += 1
    
    ratings = {
        5: "🌟 EXCELLENT",
        4: "⭐ VERY GOOD", 
        3: "✅ GOOD",
        2: "⚠️  FAIR",
        1: "❌ POOR",
        0: "💀 VERY POOR"
    }
    
    print(f"Overall Rating:      {ratings.get(rating_score, '❓ UNKNOWN')} ({rating_score}/5)")
    
    print("\n" + "="*80)

# Display the performance report
print_performance_report()

## 4. Comprehensive Visualizations

In [ ]:
# Set up the plotting environment
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

def create_equity_curve_plot():
    """
    Create equity curve and drawdown visualization.
    """
    if equity_curve.empty:
        print("No equity curve data available")
        return
        
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # Equity curve
    ax1.plot(equity_curve['timestamp'], equity_curve['equity'], linewidth=2, color='blue', label='Equity')
    ax1.set_title('Equity Curve', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Balance ($)', fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Add starting balance line
    if portfolio_metrics:
        ax1.axhline(y=portfolio_metrics['starting_balance'], color='gray', linestyle='--', alpha=0.7, label='Starting Balance')
        ax1.legend()
    
    # Drawdown
    if 'drawdown_series' in drawdown_metrics:
        ax2.fill_between(equity_curve['timestamp'], drawdown_metrics['drawdown_series'], 0, 
                        color='red', alpha=0.3, label='Drawdown')
        ax2.plot(equity_curve['timestamp'], drawdown_metrics['drawdown_series'], 
                color='red', linewidth=1)
        ax2.set_title('Drawdown', fontsize=16, fontweight='bold')
        ax2.set_ylabel('Drawdown (%)', fontsize=12)
        ax2.set_xlabel('Date', fontsize=12)
        ax2.grid(True, alpha=0.3)
        ax2.legend()
    
    plt.tight_layout()
    plt.show()

create_equity_curve_plot()

In [ ]:
def create_trade_analysis_plots():
    """
    Create comprehensive trade analysis visualizations.
    """
    if trades_df.empty:
        print("No trade data available")
        return
        
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # 1. P&L Distribution
    axes[0, 0].hist(trades_df['pnl'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].axvline(trades_df['pnl'].mean(), color='red', linestyle='--', label=f'Mean: ${trades_df["pnl"].mean():.2f}')
    axes[0, 0].set_title('P&L Distribution', fontweight='bold')
    axes[0, 0].set_xlabel('P&L ($)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. P&L by Symbol
    symbol_pnl = trades_df.groupby('symbol')['pnl'].sum().sort_values(ascending=False)
    colors = ['green' if x > 0 else 'red' for x in symbol_pnl.values]
    axes[0, 1].bar(symbol_pnl.index, symbol_pnl.values, color=colors, alpha=0.7)
    axes[0, 1].set_title('P&L by Symbol', fontweight='bold')
    axes[0, 1].set_xlabel('Symbol')
    axes[0, 1].set_ylabel('Total P&L ($)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Trade Count by Symbol
    symbol_counts = trades_df['symbol'].value_counts()
    axes[0, 2].pie(symbol_counts.values, labels=symbol_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0, 2].set_title('Trade Distribution by Symbol', fontweight='bold')
    
    # 4. Win/Loss Ratio
    trade_results = trades_df['trade_result'].value_counts()
    colors_result = ['green', 'red', 'gray']
    axes[1, 0].pie(trade_results.values, labels=trade_results.index, autopct='%1.1f%%', 
                   colors=colors_result, startangle=90)
    axes[1, 0].set_title('Win/Loss Distribution', fontweight='bold')
    
    # 5. Trades by Hour of Day
    hourly_trades = trades_df['hour'].value_counts().sort_index()
    axes[1, 1].bar(hourly_trades.index, hourly_trades.values, alpha=0.7, color='orange')
    axes[1, 1].set_title('Trades by Hour of Day', fontweight='bold')
    axes[1, 1].set_xlabel('Hour')
    axes[1, 1].set_ylabel('Number of Trades')
    axes[1, 1].grid(True, alpha=0.3)
    
    # 6. Cumulative P&L over Time
    trades_sorted = trades_df.sort_values('timestamp')
    cumulative_pnl = trades_sorted['pnl'].cumsum()
    axes[1, 2].plot(trades_sorted['timestamp'], cumulative_pnl, linewidth=2, color='purple')
    axes[1, 2].set_title('Cumulative P&L over Time', fontweight='bold')
    axes[1, 2].set_xlabel('Date')
    axes[1, 2].set_ylabel('Cumulative P&L ($)')
    axes[1, 2].grid(True, alpha=0.3)
    axes[1, 2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

create_trade_analysis_plots()

In [ ]:
def create_portfolio_analysis_plots():
    """
    Create portfolio performance visualizations.
    """
    if portfolio_df.empty:
        print("No portfolio data available")
        return
        
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. Portfolio Balance over Time
    axes[0, 0].plot(portfolio_df['timestamp'], portfolio_df['total_balance'], 
                    linewidth=2, color='blue', label='Total Balance')
    axes[0, 0].set_title('Portfolio Balance over Time', fontweight='bold')
    axes[0, 0].set_ylabel('Balance ($)')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Unrealized P&L over Time
    axes[0, 1].plot(portfolio_df['timestamp'], portfolio_df['unrealized_pnl'], 
                    linewidth=2, color='orange', label='Unrealized P&L')
    axes[0, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.7)
    axes[0, 1].set_title('Unrealized P&L over Time', fontweight='bold')
    axes[0, 1].set_ylabel('Unrealized P&L ($)')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].legend()
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Open Positions Count
    axes[1, 0].plot(portfolio_df['timestamp'], portfolio_df['open_positions_count'], 
                    linewidth=2, color='green', marker='o', markersize=2, label='Open Positions')
    axes[1, 0].set_title('Open Positions over Time', fontweight='bold')
    axes[1, 0].set_ylabel('Number of Positions')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].legend()
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. Daily Returns Distribution
    daily_returns = portfolio_df['balance_change_pct'].dropna()
    if not daily_returns.empty:
        axes[1, 1].hist(daily_returns, bins=30, alpha=0.7, color='purple', edgecolor='black')
        axes[1, 1].axvline(daily_returns.mean(), color='red', linestyle='--', 
                          label=f'Mean: {daily_returns.mean():.3f}%')
        axes[1, 1].set_title('Daily Returns Distribution', fontweight='bold')
        axes[1, 1].set_xlabel('Daily Return (%)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

create_portfolio_analysis_plots()

## 5. Advanced Analysis

In [ ]:
def create_advanced_analysis():
    """
    Create advanced trading analysis.
    """
    if trades_df.empty:
        print("No trade data available for advanced analysis")
        return
        
    print("📈 ADVANCED TRADING ANALYSIS")
    print("=" * 50)
    
    # Trade performance by day of week
    print("\n📅 Performance by Day of Week:")
    day_performance = trades_df.groupby('day_of_week').agg({
        'pnl': ['count', 'sum', 'mean'],
        'trade_result': lambda x: (x == 'Win').sum() / len(x) * 100
    }).round(2)
    
    day_performance.columns = ['Trades', 'Total_PnL', 'Avg_PnL', 'Win_Rate']
    print(day_performance)
    
    # Performance by symbol
    print("\n💰 Performance by Symbol:")
    symbol_performance = trades_df.groupby('symbol').agg({
        'pnl': ['count', 'sum', 'mean'],
        'trade_result': lambda x: (x == 'Win').sum() / len(x) * 100,
        'fees': 'sum'
    }).round(2)
    
    symbol_performance.columns = ['Trades', 'Total_PnL', 'Avg_PnL', 'Win_Rate', 'Total_Fees']
    print(symbol_performance.sort_values('Total_PnL', ascending=False))
    
    # Best and worst trades
    print("\n🏆 Top 5 Best Trades:")
    best_trades = trades_df.nlargest(5, 'pnl')[['timestamp', 'symbol', 'direction', 'pnl', 'entry_reason']]
    print(best_trades.to_string(index=False))
    
    print("\n💥 Top 5 Worst Trades:")
    worst_trades = trades_df.nsmallest(5, 'pnl')[['timestamp', 'symbol', 'direction', 'pnl', 'exit_reason']]
    print(worst_trades.to_string(index=False))
    
    # Monthly performance summary
    if len(trades_df) > 0:
        trades_df['month'] = trades_df['timestamp'].dt.to_period('M')
        monthly_performance = trades_df.groupby('month').agg({
            'pnl': ['count', 'sum', 'mean'],
            'trade_result': lambda x: (x == 'Win').sum() / len(x) * 100
        }).round(2)
        
        monthly_performance.columns = ['Trades', 'Total_PnL', 'Avg_PnL', 'Win_Rate']
        print("\n📊 Monthly Performance Summary:")
        print(monthly_performance)

create_advanced_analysis()

## 6. Export Results and Summary

In [ ]:
def export_analysis_results():
    """
    Export analysis results to files.
    """
    try:
        # Create exports directory
        export_dir = Path("../exports")
        export_dir.mkdir(exist_ok=True)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Export metrics to JSON
        all_metrics = {
            'generated_at': datetime.now().isoformat(),
            'basic_metrics': basic_metrics,
            'portfolio_metrics': portfolio_metrics,
            'drawdown_metrics': {k: v for k, v in drawdown_metrics.items() if k != 'drawdown_series'},
            'sharpe_ratio': sharpe_ratio
        }
        
        metrics_file = export_dir / f"performance_metrics_{timestamp}.json"
        
        import json
        with open(metrics_file, 'w') as f:
            json.dump(all_metrics, f, indent=2, default=str)
        
        # Export equity curve
        if not equity_curve.empty:
            equity_file = export_dir / f"equity_curve_{timestamp}.csv"
            equity_curve.to_csv(equity_file, index=False)
            print(f"✅ Equity curve exported to: {equity_file}")
        
        print(f"✅ Performance metrics exported to: {metrics_file}")
        print(f"\n📁 All exports saved to: {export_dir.absolute()}")
        
    except Exception as e:
        print(f"❌ Error exporting results: {e}")

export_analysis_results()

# Final summary
print("\n" + "="*80)
print("                    ANALYSIS COMPLETE")
print("="*80)
print("\n✅ Performance analysis notebook execution completed successfully!")
print("\n📋 Generated Analysis:")
print("   • Comprehensive performance metrics")
print("   • Equity curve and drawdown analysis")
print("   • Trade distribution visualizations")
print("   • Portfolio performance tracking")
print("   • Advanced trading statistics")
print("   • Exported results for further use")
print("\n🎯 Next Steps:")
print("   • Review performance metrics for strategy optimization")
print("   • Analyze losing trades for pattern identification")
print("   • Consider position sizing adjustments based on volatility")
print("   • Implement risk management improvements if needed")